In [1]:
import pandas as pd
import numpy as np
import re
import copy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)  # Displays all the text for each value in a column
pd.set_option('display.float_format', '{:,.2f}'.format)

def format_bigquery_column_names(nftfi):
    # Remove special charachters
    nftfi.columns = [re.sub(r'[^a-zA-Z0-9\s]+', '', column) for column in nftfi.columns]
    # Remove spaces at beginning and end
    nftfi.columns = nftfi.columns.str.strip()
    # Replace spaces with underscore
    nftfi.columns = nftfi.columns.str.replace(' ', '_')
    # Define a function to convert camel-case to kebab-case
    def camel_to_kebab(s):
        # Replace consecutive capital letters with a single lowercase letter
        s = re.sub(r'(?<=[a-z])(?=[A-Z])', '_', s)
        # Convert remaining camel-case string to kebab-case
        s = re.sub(r'(?<!^)(?<!_)(?=[A-Z])(?![A-Z])', '_', s).lower()
        return s
    # Apply the function to all column names
    nftfi.columns = nftfi.columns.map(camel_to_kebab)
    return nftfi



## Load tables

In [2]:
# try: 
#     mq = pd.read_csv('MQ_from_metaquants_telegram.csv')
# except FileNotFoundError:
#     mq = pd.read_csv('analysis/metaquants_analysis/MQ_from_metaquants_telegram.csv')

In [3]:
file_name = 'mq_nftfi_data_5'  # 'metaquants_loans'
try: 
    mq = pd.read_csv(f'data/{file_name}.csv')
except FileNotFoundError:
    mq = pd.read_csv(f'../../data/{file_name}.csv')

In [4]:
mq = mq.sort_values(by='block_timestamp', ascending=False)
display(mq.head())
display(mq.protocol.unique())
mq = mq.loc[mq['protocol'] == 'nftfi']
display(mq.protocol.unique())

,Unnamed: 0,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,method,protocol,amt_in_usd,roll_over,block_number
44212,44212,0x78c0b79cab5a183deb80a8d573b607eb941e095a6bdcc5fc1185523d331c0da3,2023-05-29 16:36:23+00:00,37286,0xfa577afe273f088d37c022375454f2d6898a40c7,0x064980edd7d43abee781f49a4e31c06df05ecccb,0.80,0.81,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-05 16:36:23+00:00,7.00,0.48,9244,0x5946aeaab44e65eb370ffaa6a7ef2218cff9b47d,borrow,nftfi,"1,516.46",NaN,17365869
44211,44211,0xa54243ee3140bc12738ee31c43a4961416979e16cee49235b47548a4d4eb8784,2023-05-29 15:57:47+00:00,37285,0x7c00c9f0e7aed440c0c730a9bd9ee4f49de20d5c,0x3e60c12997d3bb062530e5f7e7f0400fbb9aeca5,8.18,8.23,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-12 15:57:47+00:00,14.00,0.14,245,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"15,504.87",NaN,17365678
44210,44210,0xf3c024a98181b441be03a93b61c653ca0e43c13d5f055eda71d52e6c7fd5f714,2023-05-29 15:47:47+00:00,37284,0x09a26345a7ed7350133934f9b2dfe93a75e01173,0x2256b007622c7bb4036ae325c5adf51e12879262,0.30,0.30,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-12 15:47:47+00:00,14.00,0.06,3631,0x3110ef5f612208724ca51f5761a69081809f03b7,borrow,nftfi,568.43,NaN,17365628
44209,44209,0x99bcfdf4d3c88626b133f93c6e7f37371cc64a06e613db1e8fa6a2593cb98cbf,2023-05-29 15:37:47+00:00,37283,0xe4fb9ca52e39b83be69e74e7fac8f807a9c16ef9,0x328eab573baae9ee939eb19cfecd4e2fffbb3549,0.03,0.03,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-28 15:37:47+00:00,30.00,0.10,1445,0xd4d871419714b778ebec2e22c7c53572b573706e,borrow,nftfi,56.84,NaN,17365579
44208,44208,0x3dae4e4a7f0ab45ae52e0307ff4cf4d00f967f36dae531715758bc38fa46b95a,2023-05-29 15:25:23+00:00,37282,0xf93f5436d997920129cc9697f0667801c577e847,0x47a0dfeb07abebd5f77aa5ffaa18faecd7686b4f,0.61,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-28 15:25:23+00:00,30.00,0.09,9774,0xccc441ac31f02cd96c153db6fd5fe0a2f4e6a68d,borrow,nftfi,"1,160.30",NaN,17365518


array(['nftfi'], dtype=object)

array(['nftfi'], dtype=object)

In [5]:
# try: 
#     nftfi = pd.read_csv('../../analytics_bot_langchain/data/nftfi/loans_with_transaction_hash.csv')
# except FileNotFoundError:
#     nftfi = pd.read_csv('analytics_bot_langchain/data/nftfi/loans_with_transaction_hash.csv')
# nftfi = format_bigquery_column_names(nftfi)
# nftfi = nftfi.rename(columns={'loan_id': 'loan_no', 'loan_date': 'date'})
# 
# # Drop nftfi gas price related columns
# for col in nftfi.columns:
#     if 'gas' in col:
#         nftfi = nftfi.drop(columns=[col],axis=1)
#     if 'fee' in col:
#         nftfi = nftfi.drop(columns=[col],axis=1)

In [6]:
try: 
    nftfi = pd.read_csv('loans_raw_loan_input_datanftfi_loans.csv')
except FileNotFoundError:
    nftfi = pd.read_csv('loans_raw_loan_input_data.csv')
raw_nftfi = copy.deepcopy(nftfi)
nftfi = format_bigquery_column_names(nftfi)
nftfi = nftfi.rename(columns={'loan_id': 'loan_no', 'loan_date': 'date'})

# Drop nftfi gas price related columns
for col in nftfi.columns:
    if 'gas' in col:
        nftfi = nftfi.drop(columns=[col],axis=1)
    if 'fee' in col:
        nftfi = nftfi.drop(columns=[col],axis=1)

/tmp/ipykernel_715561/3619570390.py:4: DtypeWarning: Columns (7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  nftfi = pd.read_csv('loans_raw_loan_input_data.csv')


### Subselect each dataset to match their start and end dates based on who has most data

In [7]:
# Check minimum and maximum dates of each table
print(f"min mq date: {mq.block_timestamp.min()}; max mq date: {mq.block_timestamp.max()}")
print(f"min nftfi date: {nftfi.date.min()}; max nftfi date: {nftfi.date.max()}")

if nftfi['date'].min() > mq['block_timestamp'].min():
    # MQ has more complete dataset
    mq = mq.loc[mq['block_timestamp'] >= nftfi['date'].min()]
else:
    # NFTfi has more complete dataset
    nftfi = nftfi.loc[nftfi['date'] >= mq['block_timestamp'].min()]

if nftfi['date'].max() > mq['block_timestamp'].max():
    # NFTfi has more complete dataset
    nftfi = nftfi.loc[nftfi['date'] <= mq['block_timestamp'].max()]
else:
    # MQ has more complete dataset
    mq = mq.loc[mq['block_timestamp'] <= nftfi['date'].max()]

print('\n\n --------------- AFTER')
# Check minimum and maximum dates of each table
print(f"min mq date: {mq.block_timestamp.min()}; max mq date: {mq.block_timestamp.max()}")
print(f"min nftfi date: {nftfi.date.min()}; max nftfi date: {nftfi.date.max()}")

min mq date: 2020-05-15 22:06:24+00:00; max mq date: 2023-05-29 16:36:23+00:00
min nftfi date: 2022-03-30 10:21:32+00:00; max nftfi date: 2023-05-30 07:02:11+00:00


 --------------- AFTER
min mq date: 2022-03-30 10:21:32+00:00; max mq date: 2023-05-29 16:36:23+00:00
min nftfi date: 2022-03-30 10:21:32+00:00; max nftfi date: 2023-05-29 16:36:23+00:00


In [8]:
print('MQ')
display(mq.head(2))
print('\n--------------------------------------------------------------------')
print('\n\nNFTfi')
display(nftfi.head(2))

MQ


,Unnamed: 0,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,method,protocol,amt_in_usd,roll_over,block_number
44212,44212,0x78c0b79cab5a183deb80a8d573b607eb941e095a6bdcc5fc1185523d331c0da3,2023-05-29 16:36:23+00:00,37286,0xfa577afe273f088d37c022375454f2d6898a40c7,0x064980edd7d43abee781f49a4e31c06df05ecccb,0.80,0.81,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-05 16:36:23+00:00,7.00,0.48,9244,0x5946aeaab44e65eb370ffaa6a7ef2218cff9b47d,borrow,nftfi,"1,516.46",NaN,17365869
44211,44211,0xa54243ee3140bc12738ee31c43a4961416979e16cee49235b47548a4d4eb8784,2023-05-29 15:57:47+00:00,37285,0x7c00c9f0e7aed440c0c730a9bd9ee4f49de20d5c,0x3e60c12997d3bb062530e5f7e7f0400fbb9aeca5,8.18,8.23,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-12 15:57:47+00:00,14.00,0.14,245,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"15,504.87",NaN,17365678



--------------------------------------------------------------------


NFTfi


,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time
0,2022-03-30 10:21:32+00:00,v2.loan.fixed-1,2022-03-30T10:21:32.000Z,2022-04-13T10:21:32.000Z,True,14.00,True,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:27:49.000Z,2022-03-13T10:48:13.000Z
1,2022-03-30 10:36:00+00:00,v2.loan.fixed-2,2022-03-30T10:36:00.000Z,2022-04-13T10:36:00.000Z,True,14.00,False,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,735355,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:36:56.000Z,NaN


In [9]:
mq['loan_id'].head(2)

44212    37286
44211    37285
Name: loan_id, dtype: int64

### Clean data

In [10]:
# clean loan IDs to try and compare them
mq['loan_id_cleaned'] = mq['loan_id'].astype(str).str.replace('-','')
nftfi['loan_no_cleaned'] = nftfi['loan_no'].str.split('-').apply(lambda x: x[-1])
nftfi

,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time,loan_no_cleaned
0,2022-03-30 10:21:32+00:00,v2.loan.fixed-1,2022-03-30T10:21:32.000Z,2022-04-13T10:21:32.000Z,True,14.00,True,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:27:49.000Z,2022-03-13T10:48:13.000Z,1
1,2022-03-30 10:36:00+00:00,v2.loan.fixed-2,2022-03-30T10:36:00.000Z,2022-04-13T10:36:00.000Z,True,14.00,False,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,735355,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:36:56.000Z,NaN,2
2,2022-03-30 10:43:05+00:00,v2.loan.fixed-3,2022-03-30T10:43:05.000Z,2022-03-30T10:50:17.000Z,True,0.01,True,10000000000000000,10000100000000000,NaN,NaN,"100,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,73.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaN,2022-03-30T10:59:51.000Z,3
3,2022-03-30 12:58:30+00:00,v2.loan.fixed-4,2022-03-30T12:58:30.000Z,2022-04-13T12:58:30.000Z,True,14.00,True,100000000000000,101000000000000,NaN,NaN,"1,000,000,000,000.00",NaN,0x47842f8b1263D880c43743FBD33D9E8dcde43a91,0x712AAc15cE295C9e503cf0D579FEF4a910644Ccf,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,184000128,False,26.07,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T13:01:53.000Z,2022-01-25T10:39:14.000Z,4
4,2022-03-30 14:19:41+00:00,v2.loan.fixed-5,2022-03-30T14:19:41.000Z,2022-04-13T14:19:41.000Z,True,14.00,False,30000000000000000000,30230137000000000000,NaN,NaN,"230,137,000,000,000,000.00",NaN,0x47842f8b1263D880c43743FBD33D9E8dcde43a91,0x712AAc15cE295C9e503cf0D579FEF4a910644Ccf,0xa7d8d9ef8D8Ce8992Df33D8b8CF4Aebabd5bD270,184000128,False,20.00,0x6B175474E89094C44Da98b954EedeAC495271d0F,2022-03-30T14:28:29.000Z,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37282,2023-05-29 15:25:23+00:00,v2-1.loan.fixed-37282,2023-05-29T15:25:23.000Z,2023-06-28T15:25:23.000Z,False,30.00,False,"612,370,000,000,000,000.00","616,899,860,273,972,608.00",NaN,NaN,4529860273972608,NaN,0x47a0DFeb07AbeBd5f77aA5ffAA18fAeCd7686b4F,0xf93f5436d997920129Cc9697F0667801C577e847,0xCcc441ac31f02cD96C153DB6fd5Fe0a2F4e6A68d,9774,True,9.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaN,NaN,37282
37283,2023-05-29 15:37:47+00:00,v2-1.loan.fixed-37283,2023-05-29T15:37:47.000Z,2023-06-28T15:37:47.000Z,False,30.00,False,"30,000,000,000,000,000.00","30,247,000,000,000,000.00",NaN,NaN,247000000000000,NaN,0x328eaB573bAAe9eE939Eb19CFecD4e2FffBB3549,0xe4fb9ca52E39B83BE69E74E7Fac8f807A9c16eF9,0xD4d871419714B778eBec2E22C7c53572b573706e,1445,True,10.02,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaN,NaN,37283
37284,2023-05-29 15:47:47+00:00,v2-1.loan.fixed-37284,2023-05-29T15:47:47.000Z,2023-06-12T15:47:47.000Z,False,14.00,False,"300,000,000,000,000,000.00","300,690,000,000,000,000.00",NaN,NaN,690000000000000,NaN,0x2256b007622C7BB4036ae325c5ADF51E12879262,0x09A26345a7ED7350133934F9B2DfE93A75E01173,0x3110EF5f612208724CA51F5761A69081809F03B7,3631,True,6.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaN,NaN,37284
37285,2023-05-29 15:57:47+00:00,v2-1.loan.fixed-37285,2023-05-29T15:57:47.000Z,2023-06-12T15:57:47.000Z,True,14.00,False,"8,183,000,000,000,000,000.00","8,225,642,712,624,640,000.00",NaN,NaN,42642712624640000,NaN,0x3e60C12997d3Bb062530e5F7e7f0400FBB9aEcA5,0x7C00c9F0E7AeD440c0C730a9bD9Ee4F49de20

In [11]:
display(mq['loan_id_cleaned'])
display(nftfi['loan_no_cleaned'])

44212    37286
44211    37285
44210    37284
44209    37283
44208    37282
         ...  
6194      6191
6193         3
6192         2
6191      6190
6190         1
Name: loan_id_cleaned, Length: 38023, dtype: object

0            1
1            2
2            3
3            4
4            5
         ...  
37282    37282
37283    37283
37284    37284
37285    37285
37286    37286
Name: loan_no_cleaned, Length: 37287, dtype: object

In [12]:
mq.shape[0], nftfi.shape[0]

(38023, 37287)

In [13]:
# Check minimum and maximum dates of each table
print(f"min mq date: {mq.block_timestamp.min()}; max mq date: {mq.block_timestamp.max()}")
print(f"min nftfi date: {nftfi.date.min()}; max nftfi date: {nftfi.date.max()}")

min mq date: 2022-03-30 10:21:32+00:00; max mq date: 2023-05-29 16:36:23+00:00
min nftfi date: 2022-03-30 10:21:32+00:00; max nftfi date: 2023-05-29 16:36:23+00:00


In [14]:
mq.loc[mq.loan_id_cleaned.duplicated()==True].sort_values('loan_id_cleaned')

,Unnamed: 0,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,method,protocol,amt_in_usd,roll_over,block_number,loan_id_cleaned
6191,6191,0xbd19dbb314cdcc2be0c0c579e639596635e77b7d5e78062e3d14ae27afa12266,2022-03-30 10:24:07+00:00,-6190,0x52ac5997d2ac830d54b4bad431ab95546d279144,0x25f9ee5543d3d815051fcbedffd283ce457f7426,"1,500.00","1,570.00",0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2022-04-29 10:24:07+00:00,30.00,0.57,462,0x3fe1a4c1481c8351e91b64d5c398b159de07cbc5,borrow,nftfi,"1,500.00",NaN,14487034,6190
6194,6194,0x504e78de6e5a1aa3b3c54f4a681a6f7866ba9c827ff04cabd4c7d986bde13dc7,2022-03-30 11:03:13+00:00,-6191,0x8854631995b99bbfca16b8ea339538950181132a,0x85db67812169378666547e03b293785d7d9a6b06,3.20,3.31,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-29 11:03:13+00:00,30.00,0.40,233,0x59468516a8259058bad1ca5f8f4bff190d30e066,borrow,nftfi,"10,878.59",NaN,14487198,6191
6196,6196,0x115b9c9355d355b35475813d63856eac4fdb71a5fcac8f603cc3ab062c034279,2022-03-30 11:58:10+00:00,-6192,0xe35e2e4624987575dfd263b124a3999e180b8b89,0xdef310bddbd0e2a524775505a134a86b6a583a6b,1.60,1.66,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-13 11:58:10+00:00,14.00,0.90,159000280,0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270,borrow,nftfi,"5,439.29",NaN,14487435,6192
6195,6195,0x85f2a7ae8b7910c9f419be5a0d09fc00485b3521d16646f29f090d4750216535,2022-03-30 11:58:10+00:00,-6193,0x8264e9e0f4cbcbbbb3f8ecaec0a625b590ae790e,0x0aff497bd016000185b1c8302fa98a88ff4a4178,3.85,3.98,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-29 11:58:10+00:00,30.00,0.41,6865,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"13,088.30",NaN,14487435,6193
6197,6197,0xccc23d459435fcf98c5ad97aac9529e79cb9559717bbc492615f66d13bccdb8c,2022-03-30 12:02:36+00:00,-6194,0xe35e2e4624987575dfd263b124a3999e180b8b89,0x4daf2c826a63d065a673bdd8b49d0f71656532a9,1.00,1.03,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-29 12:02:36+00:00,30.00,0.40,2093810144,0xbc17cd7f1a58bda5d6181618090813b3050416b5,borrow,nftfi,"3,409.29",NaN,14487450,6194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13843,13843,0xb126f84a3ab9883e582e5b6c063014832e266a6bb6cb0151ec28d11ed0ca1647,2022-07-13 16:53:41+00:00,6923,0xc4ad58f2bf6b55276d263cc5dcd75660f5c3aa97,0x2c35b3d207f745ad5b17d88328286c49a0ce4d4f,0.75,0.78,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-08-12 16:53:41+00:00,30.00,0.50,328000243,0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270,borrow,nftfi,794.46,NaN,15135449,6923
13844,13844,0x0eecbc9e2a4541772600be926dfd473cc8a641fe29d231b2fddb525d99329047,2022-07-13 17:49:31+00:00,6924,0x8eea1d4c3c6d29c2e2b312fb0e7d4c81af87dfde,0x0e73f9e0a5f0bf8fe453bb51e144700a099fef69,8.85,8.97,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-07-20 17:49:31+00:00,7.00,0.72,5111,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,borrow,nftfi,"9,489.40",NaN,15135686,6924
13845,13845,0x3f81105506bb9c77f6c35fa93901ba4b744657f83bc4244880459b0d48f44e5b,2022-07-13 17:57:08+00:00,6925,0xb085298c11413020faa6b5d2e3fd23fdc0dff19b,0x3a96ce3a88e90bd6b480fa99952ddb3e2394215d,1.90,1.95,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-08-12 17:57:08+00:00,30.00,0.35,51834,0x34d85c9cdeb23fa97cb08333b511ac86e1c4e258,borrow,nftfi,"2,037.96",NaN,15135731,6925
13846,13846,0x52c3e0a9b7c2737e8fade18ced3ca2f5f59a8867be646c97f60f7175cc9f5e26,2022-07-13 18:20:24+00:00,6926,0x3203e254149d9ce8bc370647ea5077efa17a4195,0x0aff497bd016000185b1c8302fa98a88ff4a4178,76.00,78.75,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-08-12 18:20:24+00:00,30.00,0.44,586,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,borrow,nftfi,"83,097.50",NaN,15135844,6926


In [15]:
nftfi.head(2)

,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time,loan_no_cleaned
0,2022-03-30 10:21:32+00:00,v2.loan.fixed-1,2022-03-30T10:21:32.000Z,2022-04-13T10:21:32.000Z,True,14.00,True,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:27:49.000Z,2022-03-13T10:48:13.000Z,1
1,2022-03-30 10:36:00+00:00,v2.loan.fixed-2,2022-03-30T10:36:00.000Z,2022-04-13T10:36:00.000Z,True,14.00,False,10000000000000000,10038000000000000,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,735355,False,9.91,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-03-30T10:36:56.000Z,NaN,2


In [16]:
print('MQ:\n')
print("Are loan_id unique? Once grouped by unique, the counts are: ")
mq_loan_id_count = pd.DataFrame(mq.groupby('loan_id').agg('count')['transaction_hash'])
mq_loan_id_count = mq_loan_id_count.reset_index()

display(mq_loan_id_count['transaction_hash'].unique())
display(mq_loan_id_count.loc[mq_loan_id_count['transaction_hash']>1])
duplicated_mq_loans = pd.merge(left=mq, right=mq_loan_id_count.loc[mq_loan_id_count['transaction_hash']>1], left_on='loan_id', right_on='loan_id')
display(duplicated_mq_loans)
display(duplicated_mq_loans.amt_in_usd.sum())
print("Are loan_id_cleaned unique? Once grouped by loan_id_cleaned, the counts are: ")
display(mq.groupby('loan_id_cleaned').agg('count')['transaction_hash'].unique())


MQ:

Are loan_id unique? Once grouped by unique, the counts are: 


array([1])

,loan_id,transaction_hash


,Unnamed: 0,transaction_hash_x,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,method,protocol,amt_in_usd,roll_over,block_number,loan_id_cleaned,transaction_hash_y


0.0

Are loan_id_cleaned unique? Once grouped by loan_id_cleaned, the counts are: 


array([1, 2])

In [17]:
print(f"duplicate lines in MQ dataset {mq.shape[0] - mq.drop_duplicates().shape[0]}")
print(f"duplicate lines in nftfi dataset {nftfi.shape[0] - nftfi.drop_duplicates().shape[0]}")

duplicate lines in MQ dataset 0
duplicate lines in nftfi dataset 1


## Clean raw NFTfi dataset

In [18]:
nftfi['date'] = pd.to_datetime(nftfi['date'], format="%Y-%m-%d %H:%M:%S%z")
nftfi['loan_start_time'] = pd.to_datetime(nftfi['loan_start_time'], format="%Y-%m-%d %H:%M:%S%z")
nftfi['loan_repaid_time'] = pd.to_datetime(nftfi['loan_repaid_time'], format="%Y-%m-%d %H:%M:%S%z")

# Remove invalid values
nftfi.replace(r"#DIV/0!", np.nan, regex=True, inplace=True)
nftfi.replace(r"#N/A", "", regex=True, inplace=True)



# Divide the currency by its respective divider
# now map missing usd_value, e.g. for stablecoin-denominated loans
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

columns_to_clean = ['loan_principal_amount', 'maximum_repayment_amount', 'maximum_repayment_amount']  # Divide loanPrincipalAmount and maximumRepaymentAmount by ETH <> WEI i.e. 1^18
for col in columns_to_clean:
    nftfi[col] = nftfi[col].astype(np.float64)
    nftfi.loc[nftfi['loan_erc20denomination'] == weth_address, col] /= 10 ** 18
    nftfi.loc[nftfi['loan_erc20denomination'] == dai_address, col] /= 10 ** 18
    nftfi.loc[nftfi['loan_erc20denomination'] == usdc_address, col] /= 10 ** 6


# nftfi['no_of_days'] = nftfi['no_of_days'].astype(np.float64)
nftfi = nftfi.rename(columns={'loan_apr': 'apr'})
nftfi['apr'] = nftfi['apr'].astype(np.float64)

# Drop last column as it is unnamed
nftfi = nftfi.drop('', axis=1, errors='ignore')

# set BOOL columns to bool type
cols = ['repaid', 'liquidated']
nftfi['repaid'] = nftfi['repaid'].fillna(False)
nftfi['repaid'] = nftfi['repaid'].replace('', False)
for col in cols:
    # nftfi[col] = nftfi[col].astype('boolean')
    nftfi[col] = nftfi[col].map({'True': True, 'False': False})
    nftfi[col] = nftfi[col].astype(bool)
nftfi['repaid'] = nftfi['repaid'].astype(bool)

In [19]:
nftfi.sort_values(by='loan_principal_amount')

,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time,loan_no_cleaned
34288,2023-04-27 18:22:11+00:00,v2-1.loan.fixed-34288,2023-04-27 18:22:11+00:00,2023-05-27T18:22:11.000Z,True,30.00,True,0.00,0.00,NaN,NaN,2582465753,NaN,0x11127F85B096B694E1b77bfCbe861f5F11f88896,0x7d6E22db7C2Ee44859061061f99E55257A5cEaC1,0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85,86571878219782098813062260535004457283669633212530069131913859874549537601361,False,31.42,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2023-04-27 18:42:47+00:00,NaN,34288
18116,2022-12-21 04:35:11+00:00,v2-1.loan.fixed-18117,2022-12-21 04:35:11+00:00,2022-12-28T04:35:11.000Z,True,7.00,True,0.00,0.00,NaN,NaN,"1,000,000,000,000.00",NaN,0x83bbee3c8a1dE9D5277a1fACe273c14Ae9aA8aF6,0x04F78cdDeC5a6cf4c0B40fBeBEB5cA49DebD44B5,0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85,33266523340797296217853586896260878768936399018396274840948193876064791248822,True,521.43,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaT,NaN,18117
33946,2023-04-25 16:26:11+00:00,v2-1.loan.fixed-33946,2023-04-25 16:26:11+00:00,2023-04-26T16:26:11.000Z,True,1.00,True,0.00,0.00,NaN,NaN,1000000000000,NaN,0xBea320E6a570905cE583446589DeaBAe65Cd7A93,0xf52a65497128702bf2d469600FEa18858235Ff82,0x1792a96E5668ad7C167ab804a100ce42395Ce54D,3705,False,456.25,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2023-04-25 16:29:11+00:00,NaN,33946
36595,2023-05-23 08:25:23+00:00,v2-1.loan.fixed-36595,2023-05-23 08:25:23+00:00,2023-08-21T08:25:23.000Z,True,90.00,True,0.00,0.00,NaN,NaN,2000000000000,NaN,0x8a32121D737Ce9c7b7b6e17cC7F10d7C2D5F8adC,0xb88E4282e7CF05B56579EA57DAE3cA570150A23b,0xF3E778F839934fC819cFA1040AabaCeCBA01e049,5456,True,8.11,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,NaT,NaN,36595
15231,2022-11-09 12:28:59+00:00,v2-1.loan.fixed-15232,2022-11-09 12:28:59+00:00,2022-11-10T12:28:59.000Z,True,1.00,True,0.00,0.00,NaN,NaN,"1,000,000,000,000.00",NaN,0x8a32121D737Ce9c7b7b6e17cC7F10d7C2D5F8adC,0xb88E4282e7CF05B56579EA57DAE3cA570150A23b,0xF3E778F839934fC819cFA1040AabaCeCBA01e049,5456,False,365.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-11-09 12:29:47+00:00,NaN,15232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912,2022-05-12 06:15:02+00:00,v2.loan.fixed-2913,2022-05-12 06:15:02+00:00,2022-06-11T06:15:02.000Z,True,30.00,True,"200,000.00",0.00,NaN,NaN,"4,931,506,849,000,017,887,232.00",NaN,0xeBF02C6E12c3eE119aBBA161C40BFEEaD0A06B15,0x4F28F214365800a323C1e4ba6EE09DaDcfd9Fc20,0xd4e4078ca3495DE5B1d4dB434BEbc5a986197782,231,False,30.00,0x6B175474E89094C44Da98b954EedeAC495271d0F,2022-06-11 05:12:11+00:00,NaN,2913
34526,2023-04-30 02:27:11+00:00,v2-1.loan.fixed-34526,2023-04-30 02:27:11+00:00,2023-05-02T02:27:11.000Z,True,2.00,True,"271,000.00",0.27,NaN,NaN,742465753,NaN,0x5CAdce51dB28AB261E50743fEB881a3F7AcA0A69,0xA9c501101C7B165090ABb56E95efe1129Df9DEAC,0x46C9CFB32627B74F91e0B5ad575c247AEc7e7847,62,False,50.00,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,2023-05-01 01:42:35+00:00,NaN,34526
9851,2022-08-29 10:24:57+00:00,v2.loan.fixed-9852,2022-08-29 10:24:57+00:00,2022-09-22T10:24:57.000Z,True,24.00,True,"350,000.00",0.00,NaN,NaN,"999,999,999,999,955,959,808.00",NaN,0xF4Fb9FA23edB32215E5284cf7dBfDB5607d51a5b,0x39fEc2e2beaB6a63c1E763D0dc4120AF60BEe39F,0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D,441,False,4.35,0x6B175474E89094C44Da98b954EedeAC495271d0F,2022-09-20 00:48:47+00:00,NaN,9852
27018,2023-02-27 14:27:47+00:00,v2-1.loan.fixed-27019,2023-02-27 14:27:47+00:00,2023-11-25T14:27:47.000Z,True,271.00,True,"500,000.00",0.00,NaN,NaN,"37,123,287,670,999,977,295,872.00",NaN,0xB82ea9DFce1AE280211f2991a09F77Fa06bbd391,0x41797b5e0398aF475422f6F33F2dc81d9a24aE33,0x9a129032F01EB4dDD764c1777c81b771C34a2fbE,40,True,10.00,0x6B175474E89094C44Da98b954EedeAC495271d0F,NaT,Na

## Enrich datasets
### Enrich NFTfi dataset with ETH price and compute usd_value of loan for weth-denominated loans

In [20]:
# Enrich dataset with ETHUSD rate i.e. ETHPrice, then fill USDValue for USDValue from loanERC20Denomination == weth_address
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'.lower()
price_df = pd.read_csv('../../analytics_bot_langchain/data/nftfi/eth_usd_input_data.csv')
price_df = price_df.rename(columns={'datetime': 'date'})
price_df['date'] = pd.to_datetime(price_df['date'], format='%Y-%m-%d %H:%M:%S%z')
price_df = price_df.drop(columns=['open', 'low', 'volume'])

nftfi['date'] = pd.to_datetime(nftfi['date'])
nftfi['loan_principal_amount'] = nftfi['loan_principal_amount'].astype(float)

nftfi = nftfi.sort_values(by='date')
price_df = price_df.sort_values(by='date')

nftfi = pd.merge_asof(nftfi, price_df, on='date', direction='backward')
nftfi['eth_price'] = nftfi['close']
nftfi = nftfi.drop(columns=['close'])
nftfi['loan_erc20denomination'] = nftfi['loan_erc20denomination'].astype(str).str.lower().str.strip()
nftfi.loc[nftfi['loan_erc20denomination'] == weth_address, 'usd_value'] = nftfi.loc[nftfi['loan_erc20denomination'] == weth_address]['loan_principal_amount'] * nftfi.loc[nftfi['loan_erc20denomination'] == weth_address]['eth_price']

# now map missing usd_value, e.g. for stablecoin-denominated loans
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
stablecoin_addresses = [usdc_address, usdt_address, dai_address]
for stablecoin_address in stablecoin_addresses:
    stablecoin_address = stablecoin_address.lower()
    nftfi.loc[nftfi['loan_erc20denomination'] == stablecoin_address, 'usd_value'] = nftfi.loc[nftfi['loan_erc20denomination'] == stablecoin_address]['loan_principal_amount']



In [21]:
print('\n\nNFTfi: \n')
print("Are loan_no unique? Once grouped by unique, the counts are: ")
nftfi_loan_no_count = pd.DataFrame(nftfi.groupby('loan_no').agg('count')['borrower'])
duplicated_nftfi_loans = pd.merge(left=nftfi, right=nftfi_loan_no_count.loc[nftfi_loan_no_count['borrower']>1], left_on='loan_no', right_on='loan_no')
display(duplicated_nftfi_loans)
display(duplicated_nftfi_loans.usd_value.sum())

display(nftfi_loan_no_count['borrower'].unique())
display(nftfi_loan_no_count.loc[nftfi_loan_no_count['borrower']>1])

print("Are loan_no_cleaned unique? Once grouped by loan_id_cleaned, the counts are: ")
display(nftfi.groupby('loan_no_cleaned').agg('count')['borrower'].unique())



NFTfi: 

Are loan_no unique? Once grouped by unique, the counts are: 


,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower_x,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time,loan_no_cleaned,high,eth_price,usd_value,borrower_y
0,2023-03-14 14:19:59+00:00,v2-1.loan.fixed-29040,2023-03-14 14:19:59+00:00,2023-04-13T14:19:59.000Z,True,30.00,True,0.35,0.00,NaN,NaN,"22,726,000,000,000,000.00",NaN,0x34a1E2E6b781763001ef7187Ba26ACb383BAed91,0xBDeD327A145B4703300E10204E6d180e1a15C844,0x942BC2d3e7a589FE5bd4A5C6eF9727DFd82F5C8a,6422,True,79.00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaT,NaN,29040,"1,740.72","1,733.65",606.78,2
1,2023-03-14 14:19:59+00:00,v2-1.loan.fixed-29040,2023-03-14 14:19:59+00:00,2023-04-13T14:19:59.000Z,True,30.00,True,0.35,0.00,NaN,NaN,"22,726,000,000,000,000.00",NaN,0x34a1E2E6b781763001ef7187Ba26ACb383BAed91,0xBDeD327A145B4703300E10204E6d180e1a15C844,0x942BC2d3e7a589FE5bd4A5C6eF9727DFd82F5C8a,6422,True,79.00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaT,NaN,29040,"1,740.72","1,733.65",606.78,2


1213.555

array([1, 2])

,borrower
loan_no,
v2-1.loan.fixed-29040,2


Are loan_no_cleaned unique? Once grouped by loan_id_cleaned, the counts are: 


array([1, 2])

### Enrich nftfi table with ERC20 denominations

In [22]:
# Check if it can be due to missing token
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

stablecoin_addresses = [usdc_address, usdt_address, dai_address, weth_address]

stablecoin_names = ['usdc', 'usdt', 'dai', 'weth']
for stablecoin_address, stablecoin_name in zip(stablecoin_addresses, stablecoin_names):
    stablecoin_address = stablecoin_address.lower()
    nftfi.loc[nftfi['loan_erc20denomination'] == stablecoin_address, 'loan_erc20denomination_name'] = stablecoin_name

In [23]:
# Confirm that the above addresses matches the one available in MQ dataset:
mq.erc20_address.unique()

array(['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
       '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
       '0x6b175474e89094c44da98b954eedeac495271d0f'], dtype=object)

#### Enrich NFTfi table with loan types

In [24]:
nftfi['loan_type'] = nftfi.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0])
nftfi['loan_type'].unique()

array(['v2.loan.fixed', 'v2-1.loan.fixed', 'v2.loan.fixed.collection'],
      dtype=object)

In [25]:
mq

,Unnamed: 0,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,method,protocol,amt_in_usd,roll_over,block_number,loan_id_cleaned
44212,44212,0x78c0b79cab5a183deb80a8d573b607eb941e095a6bdcc5fc1185523d331c0da3,2023-05-29 16:36:23+00:00,37286,0xfa577afe273f088d37c022375454f2d6898a40c7,0x064980edd7d43abee781f49a4e31c06df05ecccb,0.80,0.81,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-05 16:36:23+00:00,7.00,0.48,9244,0x5946aeaab44e65eb370ffaa6a7ef2218cff9b47d,borrow,nftfi,"1,516.46",NaN,17365869,37286
44211,44211,0xa54243ee3140bc12738ee31c43a4961416979e16cee49235b47548a4d4eb8784,2023-05-29 15:57:47+00:00,37285,0x7c00c9f0e7aed440c0c730a9bd9ee4f49de20d5c,0x3e60c12997d3bb062530e5f7e7f0400fbb9aeca5,8.18,8.23,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-12 15:57:47+00:00,14.00,0.14,245,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"15,504.87",NaN,17365678,37285
44210,44210,0xf3c024a98181b441be03a93b61c653ca0e43c13d5f055eda71d52e6c7fd5f714,2023-05-29 15:47:47+00:00,37284,0x09a26345a7ed7350133934f9b2dfe93a75e01173,0x2256b007622c7bb4036ae325c5adf51e12879262,0.30,0.30,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-12 15:47:47+00:00,14.00,0.06,3631,0x3110ef5f612208724ca51f5761a69081809f03b7,borrow,nftfi,568.43,NaN,17365628,37284
44209,44209,0x99bcfdf4d3c88626b133f93c6e7f37371cc64a06e613db1e8fa6a2593cb98cbf,2023-05-29 15:37:47+00:00,37283,0xe4fb9ca52e39b83be69e74e7fac8f807a9c16ef9,0x328eab573baae9ee939eb19cfecd4e2fffbb3549,0.03,0.03,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-28 15:37:47+00:00,30.00,0.10,1445,0xd4d871419714b778ebec2e22c7c53572b573706e,borrow,nftfi,56.84,NaN,17365579,37283
44208,44208,0x3dae4e4a7f0ab45ae52e0307ff4cf4d00f967f36dae531715758bc38fa46b95a,2023-05-29 15:25:23+00:00,37282,0xf93f5436d997920129cc9697f0667801c577e847,0x47a0dfeb07abebd5f77aa5ffaa18faecd7686b4f,0.61,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-28 15:25:23+00:00,30.00,0.09,9774,0xccc441ac31f02cd96c153db6fd5fe0a2f4e6a68d,borrow,nftfi,"1,160.30",NaN,17365518,37282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6194,6194,0x504e78de6e5a1aa3b3c54f4a681a6f7866ba9c827ff04cabd4c7d986bde13dc7,2022-03-30 11:03:13+00:00,-6191,0x8854631995b99bbfca16b8ea339538950181132a,0x85db67812169378666547e03b293785d7d9a6b06,3.20,3.31,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-29 11:03:13+00:00,30.00,0.40,233,0x59468516a8259058bad1ca5f8f4bff190d30e066,borrow,nftfi,"10,878.59",NaN,14487198,6191
6193,6193,0x7d7682a9827b44571da3bdc1f7342d9ad3c985e7aedba428bbf3a2f506306cb6,2022-03-30 10:43:05+00:00,3,0x3e3b47b5d433be4708a6bd524facd48bb54d10b1,0xd79275564ccad8e679cbbb5e1c2facd46643a372,0.01,0.01,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-03-30 10:50:17+00:00,0.01,0.73,1000323,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,33.89,NaN,14487122,3
6192,6192,0x9d783e3bbde1663ffa2dcc0bf5eabeccbe3f58475938c42e52f792f8eb85eddf,2022-03-30 10:36:00+00:00,2,0x3e3b47b5d433be4708a6bd524facd48bb54d10b1,0xd79275564ccad8e679cbbb5e1c2facd46643a372,0.01,0.01,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-13 10:36:00+00:00,14.00,0.10,735355,0x06012c8cf97bead5deae237070f9587f8e7a266d,borrow,nftfi,33.89,NaN,14487091,2
6191,6191,0xbd19dbb314cdcc2be0c0c579e639596635e77b7d5e78062e3d14ae27afa12266,2022-03-30 10:24:07+00:00,-6190,0x52ac5997d2ac830d54b4bad431ab95546d279144,0x25f9ee5543d3d815051fcbedffd283ce457f7426,"1,500.00","1,570.00",0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2022-04-29 10:24:07+00:00,30.00,0.57,462,0x3fe1a4c1481c8351e91b64d5c398b159de07cbc5,borrow,nftfi,"1,500.00",NaN,14487034,6190


In [26]:
# enrich dataset with collection name, blockchain
try:
    collection_names_df = pd.read_csv('analytics_bot_langchain/data/nft_address_name_symbol/nft_address_name_symbol.csv')
except FileNotFoundError:
    collection_names_df = pd.read_csv('../../analytics_bot_langchain/data/nft_address_name_symbol/nft_address_name_symbol.csv')
collection_names_df = collection_names_df.rename(columns={'contract_address': 'collection_address', 'name': 'collection_name'})
collection_names_df['collection_address'] = collection_names_df['collection_address'].astype(str).str.lower().str.strip()
collection_names_df = collection_names_df.drop(columns=['symbol', 'standard', 'blockchain'])
display(collection_names_df.head(2))
mq['collection_address'] = mq['collection_address'].astype(str).str.lower().str.strip()

mq_existing_cols = mq.columns
mq = pd.merge(left=mq, right=collection_names_df, on='collection_address', how='left')
new_columns_order = ['collection_name'] + list(mq_existing_cols)
mq = mq[new_columns_order]

,collection_address,collection_name
0,0x3b37270c332b5c6cf6aac0103d9d896f5dcafb1d,Riia the Girls
1,0x668d40fb53871aa139ae306bca88f00bb8c475fb,Simpin Santa


## Check USD value discrepancies for matching loans on transaction hash

In [27]:
display(nftfi.head(2))
display(mq.head(2))

mq['loan_id'] = mq['loan_id'].astype(int)
nftfi['loan_no_cleaned'] = nftfi['loan_no_cleaned'].astype(int)


nftfi.columns = [f'nftfi_{col}' for col in nftfi.columns]
mq.columns = [f'mq_{col}' for col in mq.columns]
full_inner = pd.merge(nftfi, mq, how='inner', left_on='nftfi_loan_no_cleaned', right_on='mq_loan_id', indicator=True)


,date,loan_no,loan_start_time,loan_due_time,repaid,no_of_days,liquidated,loan_principal_amount,maximum_repayment_amount,ethprice,usdvalue,interest,interest_in_usd,lender,borrower,nft_collateral_contract,nft_collateral_id,active,apr,loan_erc20denomination,loan_repaid_time,loan_liquidation_time,loan_no_cleaned,high,eth_price,usd_value,loan_erc20denomination_name,loan_type
0,2022-03-30 10:21:32+00:00,v2.loan.fixed-1,2022-03-30 10:21:32+00:00,2022-04-13T10:21:32.000Z,True,14.00,True,0.01,0.00,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,9.91,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-03-30 10:27:49+00:00,2022-03-13T10:48:13.000Z,1,"3,396.32","3,394.93",33.95,weth,v2.loan.fixed
1,2022-03-30 10:36:00+00:00,v2.loan.fixed-2,2022-03-30 10:36:00+00:00,2022-04-13T10:36:00.000Z,True,14.00,True,0.01,0.00,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,735355,False,9.91,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-03-30 10:36:56+00:00,NaN,2,"3,398.36","3,397.60",33.98,weth,v2.loan.fixed


,collection_name,Unnamed: 0,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,method,protocol,amt_in_usd,roll_over,block_number,loan_id_cleaned
0,Creepz by OVERLORD,44212,0x78c0b79cab5a183deb80a8d573b607eb941e095a6bdcc5fc1185523d331c0da3,2023-05-29 16:36:23+00:00,37286,0xfa577afe273f088d37c022375454f2d6898a40c7,0x064980edd7d43abee781f49a4e31c06df05ecccb,0.80,0.81,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-05 16:36:23+00:00,7.00,0.48,9244,0x5946aeaab44e65eb370ffaa6a7ef2218cff9b47d,borrow,nftfi,"1,516.46",NaN,17365869,37286
1,Art Blocks,44211,0xa54243ee3140bc12738ee31c43a4961416979e16cee49235b47548a4d4eb8784,2023-05-29 15:57:47+00:00,37285,0x7c00c9f0e7aed440c0c730a9bd9ee4f49de20d5c,0x3e60c12997d3bb062530e5f7e7f0400fbb9aeca5,8.18,8.23,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-12 15:57:47+00:00,14.00,0.14,245,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"15,504.87",NaN,17365678,37285


In [28]:
usd_diff = full_inner[['mq_amt_in_usd','nftfi_usd_value', 'nftfi_loan_no_cleaned']]# .dropna()
display(usd_diff.head(2))
display(usd_diff.tail(2))
usd_diff['usd_diff_nftfi_minus_mq'] = usd_diff['nftfi_usd_value'] - usd_diff['mq_amt_in_usd']
display(usd_diff.tail(2))

,mq_amt_in_usd,nftfi_usd_value,nftfi_loan_no_cleaned
0,33.89,33.95,1
1,33.89,33.98,2


,mq_amt_in_usd,nftfi_usd_value,nftfi_loan_no_cleaned
37298,"15,504.87","14,944.86",37285
37299,"1,516.46","1,465.63",37286


/tmp/ipykernel_715561/4014243634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usd_diff['usd_diff_nftfi_minus_mq'] = usd_diff['nftfi_usd_value'] - usd_diff['mq_amt_in_usd']


,mq_amt_in_usd,nftfi_usd_value,nftfi_loan_no_cleaned,usd_diff_nftfi_minus_mq
37298,"15,504.87","14,944.86",37285,-560.02
37299,"1,516.46","1,465.63",37286,-50.83


In [29]:
display(pd.DataFrame(usd_diff.min().values).T)
# nftfi.loc[nftfi['nftfi_transaction_hash']=='0x7c7675a6613ddaded0b187314c2edd3ad3c5428a7a14e92830b74e03fcaf7210'].T
# display(pd.DataFrame(usd_diff.min().T['nftfi_transaction_hash'])

,0,1,2,3
0,0.00,0.00,1.00,"-5,714.10"


In [30]:
pd.DataFrame(usd_diff.usd_diff_nftfi_minus_mq.describe())

,usd_diff_nftfi_minus_mq
count,"37,300.00"
mean,2.91
std,146.34
min,"-5,714.10"
25%,-2.94
50%,0.00
75%,3.35
max,"8,207.66"


# Check per collection borrow volume: all time, monthly, daily etc

In [31]:
start = '2020-04-01'
end = '2024-05-01'

april_mq = mq.loc[mq['mq_block_timestamp'] >= start].loc[mq['mq_block_timestamp'] < end]
display(april_mq.shape)
april_nftfi = nftfi.loc[nftfi['nftfi_date'] >= start].loc[nftfi['nftfi_date'] < end]
display(april_nftfi.shape)


april_full_outer = pd.merge(april_nftfi, april_mq, how='outer', left_on='nftfi_loan_no_cleaned', right_on='mq_loan_id', indicator=True)

april_full_outer_without_nftfi_na = april_full_outer.dropna(subset=['nftfi_usd_value'])
april_full_outer_without_nftfi_na.head(2)

(38037, 22)

(37287, 28)

,nftfi_date,nftfi_loan_no,nftfi_loan_start_time,nftfi_loan_due_time,nftfi_repaid,nftfi_no_of_days,nftfi_liquidated,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_ethprice,nftfi_usdvalue,nftfi_interest,nftfi_interest_in_usd,nftfi_lender,nftfi_borrower,nftfi_nft_collateral_contract,nftfi_nft_collateral_id,nftfi_active,nftfi_apr,nftfi_loan_erc20denomination,nftfi_loan_repaid_time,nftfi_loan_liquidation_time,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_Unnamed: 0,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_method,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_loan_id_cleaned,_merge
0,2022-03-30 10:21:32+00:00,v2.loan.fixed-1,2022-03-30 10:21:32+00:00,2022-04-13T10:21:32.000Z,True,14.00,True,0.01,0.00,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,9.91,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-03-30 10:27:49+00:00,2022-03-13T10:48:13.000Z,1.00,"3,396.32","3,394.93",33.95,weth,v2.loan.fixed,Art Blocks,"6,190.00",0x166d57a868ae6c3e34c192546aedc243eda0c7ba6685b741f655e3329e2b6e29,2022-03-30 10:21:32+00:00,1.00,0x3e3b47b5d433be4708a6bd524facd48bb54d10b1,0xd79275564ccad8e679cbbb5e1c2facd46643a372,0.01,0.01,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-13 10:21:32+00:00,14.00,0.10,1000323,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,33.89,NaN,"14,487,024.00",1,both
1,2022-03-30 10:36:00+00:00,v2.loan.fixed-2,2022-03-30 10:36:00+00:00,2022-04-13T10:36:00.000Z,True,14.00,True,0.01,0.00,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x06012c8cf97BEaD5deAe237070F9587f8E7A266d,735355,False,9.91,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-03-30 10:36:56+00:00,NaN,2.00,"3,398.36","3,397.60",33.98,weth,v2.loan.fixed,NaN,"6,192.00",0x9d783e3bbde1663ffa2dcc0bf5eabeccbe3f58475938c42e52f792f8eb85eddf,2022-03-30 10:36:00+00:00,2.00,0x3e3b47b5d433be4708a6bd524facd48bb54d10b1,0xd79275564ccad8e679cbbb5e1c2facd46643a372,0.01,0.01,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-13 10:36:00+00:00,14.00,0.10,735355,0x06012c8cf97bead5deae237070f9587f8e7a266d,borrow,nftfi,33.89,NaN,"14,487,091.00",2,both


In [32]:
april_full_outer_without_nftfi_na = april_full_outer_without_nftfi_na.groupby('mq_collection_name').agg(sum)[['nftfi_usd_value', 'mq_amt_in_usd']]
april_full_outer_without_nftfi_na['nftfi_minus_mq'] = april_full_outer_without_nftfi_na['nftfi_usd_value'] - april_full_outer_without_nftfi_na['mq_amt_in_usd']
april_full_outer_without_nftfi_na['abs_nftfi_minus_mq'] = april_full_outer_without_nftfi_na['nftfi_minus_mq'].abs()
april_full_outer_without_nftfi_na.head(5)

,nftfi_usd_value,mq_amt_in_usd,nftfi_minus_mq,abs_nftfi_minus_mq
mq_collection_name,,,,
"""""MOAR"""" by Joan Cornella","1,638.18","1,633.59",4.59,4.59
0N1 Force,"132,287.83","132,413.41",-125.58,125.58
0xmons,"124,984.77","125,199.34",-214.57,214.57
10KTF,355.15,354.46,0.70,0.70
10KTF Gucci Grail,"19,560.05","19,567.40",-7.35,7.35


In [33]:
april_full_outer_without_nftfi_na.describe()

,nftfi_usd_value,mq_amt_in_usd,nftfi_minus_mq,abs_nftfi_minus_mq
count,282.00,282.00,282.00,282.00
mean,"1,104,705.81","1,104,319.87",385.95,545.99
std,"5,994,591.38","5,993,467.09","1,941.75","1,902.81"
min,9.55,9.52,"-2,100.53",0.00
25%,"6,529.70","6,530.44",-9.76,6.25
50%,"35,098.92","35,115.78",3.38,28.46
75%,"210,990.57","210,786.55",70.03,167.75
max,"74,286,314.17","74,277,168.14","15,741.77","15,741.77"


In [34]:
pd.options.plotting.backend = "plotly"
april_full_outer_without_nftfi_na.plot(x=april_full_outer_without_nftfi_na.index, y=april_full_outer_without_nftfi_na.nftfi_minus_mq)

In [35]:
april_full_outer_without_nftfi_na.sort_values(by='abs_nftfi_minus_mq', ascending=False)

,nftfi_usd_value,mq_amt_in_usd,nftfi_minus_mq,abs_nftfi_minus_mq
mq_collection_name,,,,
MutantApeYachtClub,"26,295,672.56","26,279,930.79","15,741.77","15,741.77"
Art Blocks Curated,"13,365,212.51","13,350,070.74","15,141.76","15,141.76"
Art Blocks,"7,619,627.81","7,606,590.98","13,036.82","13,036.82"
Otherdeed for Otherside,"11,398,336.06","11,388,889.02","9,447.04","9,447.04"
Wrapped Cryptopunks,"74,286,314.17","74,277,168.14","9,146.03","9,146.03"
Bored Ape Yacht Club,"55,730,570.19","55,722,643.74","7,926.46","7,926.46"
Moonbirds,"6,016,528.03","6,009,413.54","7,114.49","7,114.49"
PROOF Collective,"669,930.00","663,021.03","6,908.97","6,908.97"
VeeFriends,"3,640,895.80","3,635,637.50","5,258.31","5,258.31"


In [36]:
april_full_outer_without_nftfi_na.sort_values(by='nftfi_minus_mq')

,nftfi_usd_value,mq_amt_in_usd,nftfi_minus_mq,abs_nftfi_minus_mq
mq_collection_name,,,,
PudgyPenguins,"3,149,173.44","3,151,273.97","-2,100.53","2,100.53"
DeadFellaz,"426,343.28","428,377.98","-2,034.70","2,034.70"
Azuki,"10,798,936.39","10,800,775.89","-1,839.50","1,839.50"
KILLABEARS,"2,878,545.95","2,880,264.27","-1,718.32","1,718.32"
NFTfi Locked Bundle,"2,469,469.18","2,471,032.45","-1,563.26","1,563.26"
IKB Cachet de Garantie,"73,683.22","75,235.17","-1,551.95","1,551.95"
Deafbeef V2,"329,580.88","330,884.68","-1,303.80","1,303.80"
CloneX,"8,287,771.48","8,289,018.27","-1,246.79","1,246.79"
Meebits,"1,561,191.10","1,562,211.23","-1,020.12","1,020.12"


In [37]:
collection_of_interest = 'Art Blocks' # 'MutantApeYachtClub'
display(april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest].describe())
display(april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest].head(3))


,nftfi_no_of_days,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_ethprice,nftfi_usdvalue,nftfi_interest_in_usd,nftfi_apr,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,mq_Unnamed: 0,mq_loan_id,mq_principal_amount,mq_repayment_amount,mq_duration_in_days,mq_apr,mq_amt_in_usd,mq_roll_over,mq_block_number
count,640.00,640.00,640.00,0.00,0.00,0.00,640.00,640.00,640.00,640.00,640.00,645.00,645.00,645.00,645.00,645.00,645.00,645.00,0.00,645.00
mean,78.32,550.17,0.00,NaN,NaN,NaN,21.98,"21,080.60","1,650.32","1,648.39","11,905.67","27,839.55","20,866.50",545.95,562.46,78.04,0.22,"11,919.65",NaN,"16,247,258.06"
std,109.17,"2,794.80",0.00,NaN,NaN,NaN,17.25,"11,076.22",348.24,348.21,"9,594.51","11,195.52","11,296.34","2,784.34","2,860.81",108.82,0.17,"9,521.60",NaN,"829,346.85"
min,0.01,0.01,0.00,NaN,NaN,NaN,0.98,1.00,"1,024.30","1,022.06",33.95,"6,190.00","-6,902.00",0.01,0.01,0.01,0.01,33.89,NaN,"14,487,024.00"
25%,30.00,5.00,0.00,NaN,NaN,NaN,9.00,"10,818.25","1,350.30","1,345.77","7,857.30","17,430.00","10,503.00",5.00,5.08,30.00,0.09,"7,896.29",NaN,"15,491,135.00"
50%,30.00,7.00,0.00,NaN,NaN,NaN,18.00,"21,282.50","1,651.04","1,650.21","11,954.44","28,206.00","21,279.00",7.00,7.63,30.00,0.18,"12,000.00",NaN,"16,418,609.00"
75%,60.00,9.60,0.00,NaN,NaN,NaN,30.00,"32,175.75","1,840.68","1,839.12","14,287.32","39,050.00","32,123.00",9.50,9.97,60.00,0.30,"14,333.68",NaN,"17,012,853.00"
max,371.00,"30,000.00",0.01,NaN,NaN,NaN,100.00,"37,285.00","3,402.04","3,398.59","105,787.50","44,211.00","37,285.00","30,000.00","30,739.73",371.00,1.00,"105,356.82",NaN,"17,365,678.00"


,nftfi_date,nftfi_loan_no,nftfi_loan_start_time,nftfi_loan_due_time,nftfi_repaid,nftfi_no_of_days,nftfi_liquidated,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_ethprice,nftfi_usdvalue,nftfi_interest,nftfi_interest_in_usd,nftfi_lender,nftfi_borrower,nftfi_nft_collateral_contract,nftfi_nft_collateral_id,nftfi_active,nftfi_apr,nftfi_loan_erc20denomination,nftfi_loan_repaid_time,nftfi_loan_liquidation_time,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_Unnamed: 0,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_method,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_loan_id_cleaned,_merge
0,2022-03-30 10:21:32+00:00,v2.loan.fixed-1,2022-03-30 10:21:32+00:00,2022-04-13T10:21:32.000Z,True,14.00,True,0.01,0.00,NaN,NaN,"38,000,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,9.91,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-03-30 10:27:49+00:00,2022-03-13T10:48:13.000Z,1.00,"3,396.32","3,394.93",33.95,weth,v2.loan.fixed,Art Blocks,"6,190.00",0x166d57a868ae6c3e34c192546aedc243eda0c7ba6685b741f655e3329e2b6e29,2022-03-30 10:21:32+00:00,1.00,0x3e3b47b5d433be4708a6bd524facd48bb54d10b1,0xd79275564ccad8e679cbbb5e1c2facd46643a372,0.01,0.01,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-13 10:21:32+00:00,14.00,0.10,1000323,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,33.89,NaN,"14,487,024.00",1,both
2,2022-03-30 10:43:05+00:00,v2.loan.fixed-3,2022-03-30 10:43:05+00:00,2022-03-30T10:50:17.000Z,True,0.01,True,0.01,0.00,NaN,NaN,"100,000,000,000.00",NaN,0xd79275564CCAd8E679cbBb5E1C2FaCD46643a372,0x3e3B47b5d433bE4708A6BD524faCd48Bb54D10b1,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,1000323,False,73.00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,NaT,2022-03-30T10:59:51.000Z,3.00,"3,402.04","3,398.59",33.99,weth,v2.loan.fixed,Art Blocks,"6,193.00",0x7d7682a9827b44571da3bdc1f7342d9ad3c985e7aedba428bbf3a2f506306cb6,2022-03-30 10:43:05+00:00,3.00,0x3e3b47b5d433be4708a6bd524facd48bb54d10b1,0xd79275564ccad8e679cbbb5e1c2facd46643a372,0.01,0.01,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-03-30 10:50:17+00:00,0.01,0.73,1000323,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,33.89,NaN,"14,487,122.00",3,both
52,2022-04-07 01:44:50+00:00,v2.loan.fixed-53,2022-04-07 01:44:50+00:00,2022-07-06T01:44:50.000Z,True,90.00,True,6.00,0.00,NaN,NaN,"739,726,000,000,000,000.00",NaN,0x2c35B3d207f745aD5B17d88328286c49a0ce4D4F,0x9cfaa61D21437a731ac905EDeB3674dE1E77C8AD,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,2472,False,50.00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-06-21 04:22:46+00:00,NaN,53.00,"3,190.90","3,183.89","19,103.34",weth,v2.loan.fixed,Art Blocks,"6,797.00",0xb5dfac62c1a49253dabb9c942eabdce919b80343385af9833244eb3456cd9105,2022-04-07 01:44:50+00:00,53.00,0x9cfaa61d21437a731ac905edeb3674de1e77c8ad,0x2c35b3d207f745ad5b17d88328286c49a0ce4d4f,6.00,6.74,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-07-06 01:44:50+00:00,90.00,0.50,2472,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"18,924.46",NaN,"14,535,904.00",53,both


In [38]:
display(april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest].loc[april_full_outer['_merge'] != 'both']['_merge'].unique())
april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest].loc[april_full_outer['_merge'] != 'both']

['right_only']
Categories (3, object): ['left_only', 'right_only', 'both']

,nftfi_date,nftfi_loan_no,nftfi_loan_start_time,nftfi_loan_due_time,nftfi_repaid,nftfi_no_of_days,nftfi_liquidated,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_ethprice,nftfi_usdvalue,nftfi_interest,nftfi_interest_in_usd,nftfi_lender,nftfi_borrower,nftfi_nft_collateral_contract,nftfi_nft_collateral_id,nftfi_active,nftfi_apr,nftfi_loan_erc20denomination,nftfi_loan_repaid_time,nftfi_loan_liquidation_time,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_Unnamed: 0,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_method,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_loan_id_cleaned,_merge
37326,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Art Blocks,"9,584.00",0x974e5a43348fa03dedfcdbf2bf3755aa2b07d4814405dd1e0860c91022f500a0,2022-05-08 15:08:03+00:00,"-6,902.00",0x68859f535a63c5306bf24c352aba0c3964ef503e,0x4cd57979977c4e4f8d242176e46bf38f8284878f,6.30,6.40,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-06-07 15:08:03+00:00,30.00,0.20,194,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"15,903.02",NaN,"14,736,775.00",6902,right_only
37484,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Art Blocks,"6,793.00",0xcebc45e68cfefa72ab059a857f7fe331aea5c3b0b02609dc225f1109f058ce71,2022-04-06 16:55:16+00:00,"-6,744.00",0xd2db35024d1817defd4bb8617bdd7f05c07a870e,0xd37d05655d8974e3d12e23cc1be06aaf4d4aece1,6.00,6.07,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-05-06 16:55:16+00:00,30.00,0.15,1623,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"19,540.55",NaN,"14,533,490.00",6744,right_only
37710,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Art Blocks,"6,524.00",0xe3fca2c16847f3c61772fa01dd943d4ae2b77ae8b5d2947df05e5cc520cb9e1c,2022-04-03 22:34:15+00:00,"-6,518.00",0x0716620e0a3da5ae123a9eb56e7874b1caaa4f33,0xd37d05655d8974e3d12e23cc1be06aaf4d4aece1,5.50,5.59,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-05-03 22:34:15+00:00,30.00,0.20,3356,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"19,409.55",NaN,"14,515,840.00",6518,right_only
37893,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Art Blocks,"6,341.00",0x3dee8d7ac887ddf79a299e8569d29c6b8d9060b0b0a2f46b1c3a6527e64e8b3b,2022-03-31 12:36:45+00:00,"-6,335.00",0x7d91d9023a4d4301089064654a53366a2f29f165,0xd37d05655d8974e3d12e23cc1be06aaf4d4aece1,4.00,4.20,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-06-29 12:36:45+00:00,90.00,0.20,7663,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"13,641.50",NaN,"14,493,981.00",6335,right_only
38036,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Art Blocks,"6,195.00",0x85f2a7ae8b7910c9f419be5a0d09fc00485b3521d16646f29f090d4750216535,2022-03-30 11:58:10+00:00,"-6,193.00",0x8264e9e0f4cbcbbbb3f8ecaec0a625b590ae790e,0x0aff497bd016000185b1c8302fa98a88ff4a4178,3.85,3.98,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-04-29 11:58:10+00:00,30.00,0.41,6865,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"13,088.30",NaN,"14,487,435.00",6193,right_only


In [39]:
# TODO: compute the implied ETH price 
april_full_outer_for_collection = april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest]
april_full_outer_for_collection['nftfi_minus_mq'] = april_full_outer_for_collection['nftfi_usd_value'] - april_full_outer_for_collection['mq_amt_in_usd']
pd.DataFrame(april_full_outer_for_collection['nftfi_minus_mq'].describe()).T

/tmp/ipykernel_715561/3243541687.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,count,mean,std,min,25%,50%,75%,max
nftfi_minus_mq,640.00,20.37,194.82,"-1,343.87",-15.49,0.00,24.01,"2,284.15"


In [40]:
april_full_outer_for_collection.dtypes
# TODO: loc for wETH address
april_full_outer_for_collection['mq_implied_eth_price'] = april_full_outer_for_collection['mq_amt_in_usd'] / april_full_outer_for_collection['mq_principal_amount']
april_full_outer_for_collection['nftfi_mq_eth_price_diff'] = april_full_outer_for_collection['nftfi_eth_price'] - april_full_outer_for_collection['mq_implied_eth_price']
highest_discrepancy_transactions = april_full_outer_for_collection[['mq_collection_name','nftfi_loan_no_cleaned','mq_transaction_hash','nftfi_minus_mq',
                                                                     'nftfi_loan_erc20denomination_name', 'nftfi_loan_principal_amount',
                                                                       'nftfi_eth_price','mq_implied_eth_price', 'nftfi_mq_eth_price_diff']].sort_values('nftfi_minus_mq')
highest_discrepancy_transactions = highest_discrepancy_transactions.dropna(subset='nftfi_minus_mq')
display(highest_discrepancy_transactions.shape[0])
highest_discrepancy_transactions

/tmp/ipykernel_715561/3575805118.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_715561/3575805118.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



640

,mq_collection_name,nftfi_loan_no_cleaned,mq_transaction_hash,nftfi_minus_mq,nftfi_loan_erc20denomination_name,nftfi_loan_principal_amount,nftfi_eth_price,mq_implied_eth_price,nftfi_mq_eth_price_diff
5125,Art Blocks,"5,126.00",0x14b5519ffeb190b46ac609b1fc3dab4546c384baa053cf3613ef1950076f592c,"-1,343.87",weth,75.00,"1,201.14","1,219.06",-17.92
37249,Art Blocks,"37,235.00",0x004817fcc157939e6eb03774e924aa9485044706c1aea63a7ef2cd5ee95a4146,-827.12,weth,10.00,"1,826.33","1,909.04",-82.71
15268,Art Blocks,"15,269.00",0x5a8f33927413fb80a01fa0c764e738d5afa35be1db903585d0e58d289251792b,-694.33,weth,15.00,"1,117.33","1,163.62",-46.29
27489,Art Blocks,"27,490.00",0xfd3aa0b6a1895d55459c347fc633d6ad44b6e51b28f60970e9187d208c9984de,-614.63,weth,7.10,"1,556.59","1,643.16",-86.57
37299,Art Blocks,"37,285.00",0xa54243ee3140bc12738ee31c43a4961416979e16cee49235b47548a4d4eb8784,-560.02,weth,8.18,"1,826.33","1,894.77",-68.44
...,...,...,...,...,...,...,...,...,...
2919,Art Blocks,"2,920.00",0x1e878fa5f9b3f4164eac1d56416851a2f9e6afb5453cffe923f6195cfb337049,950.95,weth,10.00,"1,965.48","1,870.39",95.09
2916,Art Blocks,"2,917.00",0x0dfb87ec885dd82ed7224cf7102c218d4b5d2e0ff20aafbe39fab3048a3511df,"1,001.95",weth,10.00,"1,970.58","1,870.39",100.19
2921,Art Blocks,"2,922.00",0x8965553927cba778f80da8888c371fd0f2cb5d00ebc011113c4baebe409df859,"1,058.57",weth,7.00,"2,021.61","1,870.39",151.22
7752,Art Blocks,"7,753.00",0xbaf9e5dd333365e006fc946962b1ef3d9824b6c5c6844db432f700a935bda2b2,"2,260.72",weth,50.00,"1,545.30","1,500.09",45.21


In [41]:
pd.DataFrame(highest_discrepancy_transactions['nftfi_minus_mq'].cumsum()).head(30)

,nftfi_minus_mq
5125,"-1,343.87"
37249,"-2,170.99"
15268,"-2,865.33"
27489,"-3,479.96"
37299,"-4,039.97"
10431,"-4,531.57"
37245,"-4,955.61"
5123,"-5,296.39"
36606,"-5,590.33"
23619,"-5,856.41"


In [42]:
# april_full_outer_for_collection.loc[april_full_outer_for_collection['mq_transaction_hash'] == '0xefd0018a270e01f4a53e0687a9791d512cbfcd723a6198608d689451a5111faa']
tsx_with_highest_discrepancy = highest_discrepancy_transactions.iloc[0]['mq_transaction_hash']
max_discrepancy_tsx = april_full_outer_for_collection.loc[april_full_outer_for_collection['mq_transaction_hash'] == tsx_with_highest_discrepancy]
max_discrepancy_tsx

,nftfi_date,nftfi_loan_no,nftfi_loan_start_time,nftfi_loan_due_time,nftfi_repaid,nftfi_no_of_days,nftfi_liquidated,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_ethprice,nftfi_usdvalue,nftfi_interest,nftfi_interest_in_usd,nftfi_lender,nftfi_borrower,nftfi_nft_collateral_contract,nftfi_nft_collateral_id,nftfi_active,nftfi_apr,nftfi_loan_erc20denomination,nftfi_loan_repaid_time,nftfi_loan_liquidation_time,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_Unnamed: 0,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_method,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_loan_id_cleaned,_merge,nftfi_minus_mq,mq_implied_eth_price,nftfi_mq_eth_price_diff
5125,2022-06-13 11:32:52+00:00,v2.loan.fixed-5126,2022-06-13 11:32:52+00:00,2022-07-13T11:32:52.000Z,True,30.00,True,75.00,0.00,NaN,NaN,"2,465,752,999,999,995,904.00",NaN,0x82C66866043593fC59b4e6630b4455780fA89E2C,0x28458F3442841dA5E4773b39286447D27EC57b59,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,5767,False,40.00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2022-06-13 12:43:58+00:00,NaN,"5,126.00","1,208.34","1,201.14","90,085.50",weth,v2.loan.fixed,Art Blocks,"12,044.00",0x14b5519ffeb190b46ac609b1fc3dab4546c384baa053cf3613ef1950076f592c,2022-06-13 11:32:52+00:00,"5,126.00",0x28458f3442841da5e4773b39286447d27ec57b59,0x82c66866043593fc59b4e6630b4455780fa89e2c,75.00,77.47,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2022-07-13 11:32:52+00:00,30.00,0.40,5767,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,borrow,nftfi,"91,429.37",NaN,"14,955,777.00",5126,both,"-1,343.87","1,219.06",-17.92


In [43]:
raw_nftfi.loc[raw_nftfi['loanNo'] == max_discrepancy_tsx['nftfi_loan_no'].values[0]]

,date,loanNo,loanStartTime,loanDueTime,repaid,noOfDays,liquidated,loanPrincipalAmount,maximumRepaymentAmount,ETHPrice,USDValue,interest,interestInUSD,lender,borrower,nftCollateralContract,nftCollateralId,active,APR,loanERC20Denomination,loanRepaidTime,loanLiquidationTime
5125,2022-06-13 11:32:52+00:00,v2.loan.fixed-5126,2022-06-13T11:32:52.000Z,2022-07-13T11:32:52.000Z,True,30.00,False,75000000000000000000,77465753000000000000,NaN,NaN,"2,465,752,999,999,995,904.00",NaN,0x82C66866043593fC59b4e6630b4455780fA89E2C,0x28458F3442841dA5E4773b39286447D27EC57b59,0x059EDD72Cd353dF5106D2B9cC5ab83a52287aC3a,5767,False,40.00,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,2022-06-13T12:43:58.000Z,NaN


## Compute April subset for each table

In [44]:
april_mq = mq.loc[mq['block_timestamp'] >= '2023-04-01'].loc[mq['block_timestamp'] < '2023-05-01']
april_mq.shape

KeyError: 'block_timestamp'

In [ ]:
april_nftfi = nftfi.loc[nftfi['date'] >= '2023-04-01'].loc[nftfi['date'] < '2023-05-01']
april_nftfi.shape

#### Check for duplicates

In [ ]:
april_nftfi.loc[april_nftfi.loan_no_cleaned.duplicated()== True]

In [ ]:
display(april_mq.loc[april_mq.loan_id_cleaned.duplicated()== True].head(5))
display(april_mq.loc[april_mq.loan_id_cleaned.duplicated()== True].shape[0])

In [ ]:
april_nftfi.loc[april_nftfi.loan_no_cleaned.duplicated()== True]

In [ ]:
display(april_mq.head(3).sort_values('loan_id_cleaned', ascending=False))
display(april_nftfi.head(3).sort_values('loan_no_cleaned', ascending=False))

In [ ]:
# Determine the set of unique IDs present in MQ and not in NFTfi
len(set(list(april_mq.loan_id_cleaned)) - set(list(april_nftfi.loan_no_cleaned)))

In [ ]:
# Determine the set of unique IDs present in NFTfi and not in MQ's
len(set(list(april_nftfi.loan_no_cleaned)) - set(list(april_mq.loan_id_cleaned)))

In [ ]:
april_mq.shape, april_nftfi.shape

In [ ]:
april_nftfi.shape[0] - april_mq.shape[0]

## Compute which Loans are available in NFTfi's and not in MQ's, and the converse, by matching on transaction hashes

In [ ]:
# how do i get all rows in df1 that are not in df2
# and all rows in df2 that are not in df1?

# Perform a full outer join on df1 and df2
full_outer = pd.merge(april_nftfi, april_mq, how='outer', on='transaction_hash', indicator=True)

# Filter out the rows that are unique to df1
unique_to_april_nftfi = full_outer[full_outer['_merge'] == 'left_only']

# Filter out the rows that are unique to df2
unique_to_april_mq = full_outer[full_outer['_merge'] == 'right_only']


In [ ]:
print(f"number of loans unique to NFTfi relative to MQ in April 2023: {unique_to_april_nftfi.shape[0]}, i.e. {100 * round(unique_to_april_nftfi.shape[0] / april_nftfi.shape[0], 2)}%")
print(f"while in April 2023 NFTfi had {april_nftfi.shape[0]} loans while mq had {april_mq.shape[0]}, i.e. for April NFTfi has {april_nftfi.shape[0] - april_mq.shape[0]} more loans")
display(unique_to_april_nftfi.shape[0])
display(unique_to_april_nftfi.head(2))

In [ ]:
print(f"number of loans unique to MQ relative to NFTfi in April 2023: {unique_to_april_mq.shape[0]}, i.e. {100 * round(unique_to_april_mq.shape[0] / april_mq.shape[0], 2)}%")
print(f"while in April 2023 NFTfi had {april_nftfi.shape[0]} loans while mq had {april_mq.shape[0]}, i.e. for April NFTfi has {april_nftfi.shape[0] - april_mq.shape[0]} more loans")

display(unique_to_april_mq.shape[0])
display(unique_to_april_mq.head(2))

In [ ]:
pd.set_option('display.float_format', '{:,.0f}'.format)
pd.DataFrame(unique_to_april_nftfi.groupby('loan_contract').agg(sum)['usd_value'])

In [ ]:
# MQ loans not in NFTfi
# unique_to_april_mq = april_mq[april_mq['loan_id_cleaned'].isin(mq_loans_not_in_nftfi)].reset_index(drop=True)
unique_to_april_mq.head(3)

In [ ]:
# We need to explore this table
unique_to_april_mq.erc20_name.unique()

## NFTfi loans not in MQ

In [ ]:
unique_to_april_nftfi

In [ ]:
# Explore the resulting set of unique_to_april_nftfi. Is there any pattern?
unique_to_april_nftfi.describe()

### Are discrepancies due to loan type which is not accounted for?

In [ ]:
# How about loan numbers, how do they vary from a naming standpoint?
# x[0]+x[1] if len(x) >2 else x[0] is such that it returns "v2-1.loan.fixed" from "v2-1.loan.fixed-32381", 
#    else returns "v2.loan.fixed.collection" from "v2.loan.fixed.collection-34479"

unique_to_april_nftfi.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0]).unique()
# it seems from the above result that [v2-1.loan.fixed', 'v2.loan.fixed.collection'] are the two loan types which are not accounted for

In [ ]:
# Is the above true as well for the whole table set?
nftfi_loans_not_in_mq = list(
        set(list(nftfi.loan_no_cleaned)) - set(list(mq.loan_id_cleaned))
    )
print(len(nftfi_loans_not_in_mq))
# How about loan numbers, how do they vary from a naming standpoint?
# x[0]+x[1] if len(x) >2 else x[0] is such that it returns "v2-1.loan.fixed" from "v2-1.loan.fixed-32381", 
#    else returns "v2.loan.fixed.collection" from "v2.loan.fixed.collection-34479"

unique_to_april_nftfi.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0]).unique()
# it seems from the above result that [v2-1.loan.fixed', 'v2.loan.fixed.collection'] are the two loan types which are not accounted for

### Is it due to missed erc20 token denomination?

In [ ]:
# Check if it can be due to missing token
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

stablecoin_addresses = [usdc_address, usdt_address, dai_address, weth_address]

stablecoin_names = ['usdc', 'usdt', 'dai', 'weth']
for stablecoin_address, stablecoin_name in zip(stablecoin_addresses, stablecoin_names):
    stablecoin_address = stablecoin_address.lower()
    unique_to_april_nftfi.loc[unique_to_april_nftfi['loan_erc20denomination'] == stablecoin_address, 'loan_erc20denomination_name'] = stablecoin_name

In [ ]:
unique_to_april_nftfi.loan_erc20denomination_name.unique()
# It does not seem to be linked to missing token.

### Let's compute loan volumes per token denomination to see if they match

In [ ]:
display(april_nftfi.head(2))
display(april_mq.head(2))

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.float_format', '{:,.2f}'.format)
display(april_nftfi.groupby('loan_erc20denomination_name').agg('sum'))

display(april_mq.groupby('erc20_name').agg('sum'))


In [ ]:

print("NFTfi:")
display(pd.DataFrame(april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['loan_principal_amount']))
print("\n\nMQ:")
display(pd.DataFrame(april_mq.groupby('erc20_name').agg('sum')['principal_amount']))

#### We see from the above results that NFTfi and MQ respective loan principal amounts grouped by the loan erc20 denomination do not match at all. How about the total dollar numbers?

In [ ]:
print("NFTfi:")
total_usd_nftfi_volume_per_ccy = pd.DataFrame(april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['usd_value'])
display(total_usd_nftfi_volume_per_ccy)
print("\n\nMQ:")
total_usd_mq_volume_per_ccy = pd.DataFrame(april_mq.groupby('erc20_name').agg('sum')['amt_in_usd'])
display(total_usd_mq_volume_per_ccy)

#### Total numbers are very off. Can that be explained with the set of loans which are not in each dataset?

In [ ]:
print("Total dollar value of loan principal for NFTfi loans which are not in MQ, for April and NFTfi:")
display(pd.DataFrame(unique_to_april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['usd_value']))
print("\n\nTotal dollar value of loan principal for MQ loans which are not inNFtfi, for April and MQ:")
display(pd.DataFrame(unique_to_april_mq.groupby('erc20_name').agg('sum')['amt_in_usd']))

In [ ]:
usd_values_of_nftfi_not_in_mq = pd.DataFrame(unique_to_april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['usd_value'])
usd_values_of_mq_not_in_nftfi = pd.DataFrame(unique_to_april_mq.groupby('erc20_name').agg('sum')['amt_in_usd'])

In [ ]:
weth_usd_amount_nftfi_not_in_mq = usd_values_of_nftfi_not_in_mq.loc['weth'].values[0]
weth_usd_amount_mq_not_in_nftfi = usd_values_of_mq_not_in_nftfi.loc['wETH'].values[0]

weth_usd_amount_total_nftfi = total_usd_nftfi_volume_per_ccy.loc['weth'].values[0]
weth_usd_amount_total_mq = total_usd_mq_volume_per_ccy.loc['wETH'].values[0]

print(weth_usd_amount_total_nftfi, weth_usd_amount_nftfi_not_in_mq, weth_usd_amount_mq_not_in_nftfi, weth_usd_amount_total_mq)

# total NFTfi missing from MQ volume, minus total MQ volume missing from NFTfi

weth_usd_amount_nftfi_not_in_mq - weth_usd_amount_mq_not_in_nftfi - weth_usd_amount_total_nftfi  + weth_usd_amount_total_mq

#### How about dollar amounts PER loan type PER ccy?

In [ ]:
usd_values_of_nftfi_not_in_mq = pd.DataFrame(unique_to_april_nftfi.groupby(['loan_erc20denomination_name','loan_type']).agg('sum')['usd_value'])
usd_values_of_nftfi_not_in_mq

In [ ]:
print("NFTfi:")
display(total_usd_nftfi_volume_per_ccy)
print("\n\nMQ:")
display(total_usd_mq_volume_per_ccy)

## For the loan IDs which do match... are they the same loans?

In [ ]:
import copy
nftfi_with_col_prefix = copy.deepcopy(nftfi)
mq_with_col_prefix = copy.deepcopy(mq)
nftfi_with_col_prefix = nftfi_with_col_prefix.rename(columns={col: 'nftfi_'+ col for col in nftfi.columns})
mq_with_col_prefix = mq_with_col_prefix.rename(columns={col: 'mq_'+ col for col in mq.columns})

matching_mq_with_nftfi = pd.merge(left=nftfi_with_col_prefix, right=mq_with_col_prefix, left_on='nftfi_loan_no_cleaned', right_on='mq_loan_id_cleaned', how='inner', suffixes=('nftfi', 'mq'))
matching_mq_with_nftfi

In [ ]:
subset_of_matching_mq_with_nftfi = matching_mq_with_nftfi[['nftfi_date', 'mq_block_timestamp', 'nftfi_usd_value', 'mq_amt_in_usd', 'nftfi_loan_no_cleaned', 'nftfi_loan_type', 'mq_loan_id_cleaned']]
subset_of_matching_mq_with_nftfi

In [ ]:
pd.options.plotting.backend = "plotly"
subset_of_matching_mq_with_nftfi['dollar_diff'] = subset_of_matching_mq_with_nftfi['nftfi_usd_value'] - subset_of_matching_mq_with_nftfi['mq_amt_in_usd']

subset_of_matching_mq_with_nftfi.nftfi_date = pd.to_datetime(subset_of_matching_mq_with_nftfi.nftfi_date)
subset_of_matching_mq_with_nftfi.mq_block_timestamp = pd.to_datetime(subset_of_matching_mq_with_nftfi.mq_block_timestamp)

subset_of_matching_mq_with_nftfi.plot.scatter(x=subset_of_matching_mq_with_nftfi.nftfi_date, y=subset_of_matching_mq_with_nftfi.dollar_diff)

In [ ]:
subset_of_matching_mq_with_nftfi['date_diff'] = subset_of_matching_mq_with_nftfi['nftfi_date'] - subset_of_matching_mq_with_nftfi['mq_block_timestamp']
subset_of_matching_mq_with_nftfi.plot.scatter(x=subset_of_matching_mq_with_nftfi.nftfi_date, y=subset_of_matching_mq_with_nftfi.date_diff)

In [ ]:
import numpy as np
subset_of_matching_mq_with_nftfi['date_diff'] = subset_of_matching_mq_with_nftfi['nftfi_date'] - subset_of_matching_mq_with_nftfi['mq_block_timestamp']
non_zero_time_delta = subset_of_matching_mq_with_nftfi.loc[subset_of_matching_mq_with_nftfi['date_diff'] > np.timedelta64(0)]
display(non_zero_time_delta)
display(non_zero_time_delta.nftfi_loan_type.unique())

print(f"min, max nftfi [{non_zero_time_delta.nftfi_date.min(), non_zero_time_delta.nftfi_date.max()}]")
print(f"min, max mq [{non_zero_time_delta.mq_block_timestamp.min(), non_zero_time_delta.mq_block_timestamp.max()}]")

print(f"\n\nTotal dollar diff for non zero time deltas (IDs not matching) [{non_zero_time_delta.dollar_diff.sum()}]")
print(f"Total dollar diff for both datasets [{subset_of_matching_mq_with_nftfi.dollar_diff.sum()}]")

In [ ]:
nftfi.date = pd.to_datetime(nftfi.date)
time_delta = pd.Timedelta(minutes=20)
datemax_minus_time_delta = non_zero_time_delta.nftfi_date.max() - time_delta
datemax_plus_time_delta = non_zero_time_delta.nftfi_date.max() + time_delta

print(non_zero_time_delta.nftfi_date.max())
nftfi.loc[nftfi.date < datemax_plus_time_delta].loc[datemax_minus_time_delta < nftfi.date].sort_values('date')

In [ ]:
non_zero_time_delta.loc[non_zero_time_delta['dollar_diff'] == 0]

### Are missing loans from MQ relative to NFTfi, bundles?

In [ ]:
display(unique_to_april_nftfi.collection_name.unique())
len(unique_to_april_nftfi.collection_name.unique())

In [ ]:
display(unique_to_april_mq.collection_address.unique()[:3])
len(unique_to_april_mq.collection_address.unique())

## How are liquidations accounted for?

In [ ]:
# TODO:
# spreadsheet of different transactions that are missing, or are more of them on MQ dataset. we need clear doc discussing which tsx are missing, which are added,
# what are the volume differences coming from.

# try and check that the eth value of those loans per transaction is the same.
# check if volumes per loanerc20 collateral are matching

## Sandbox

In [ ]:
try: 
    volume = pd.read_csv('analysis/nftfi_cluster_data/3month_200apr_with_borrow_volume.csv')
    base_data = pd.read_csv('analysis/nftfi_cluster_data/20230522_CADLabs_NFTfi NFT collection APR ranges_vDraft - past 3M; loan count 20; APRs 200; 5 clusters.csv')
except FileNotFoundError:
    volume = (
        pd.read_csv('nftfi_cluster_data/3month_200apr_with_borrow_volume.csv')
        .drop(columns=['mean', 'median', 'std', 'count'])
    )
    base_data = (
        pd.read_csv('nftfi_cluster_data/20230522_CADLabs_NFTfi NFT collection APR ranges_vDraft - past 3M; loan count 20; APRs 200; 5 clusters.csv')
        .drop(columns=['Unnamed: 0', 'Unnamed: 7', 'Unnamed: 8'])
    )

In [ ]:
display(volume.head(2))
display(base_data.head(2))

In [ ]:
nftfi = pd.merge(left=base_data, right=volume, on='collection_name')
nftfi 
# list(nftfi['borrow_volume'].values)